Code to extract and save $\Delta F / F$ for whole making whole brain maps

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import copy
import glob
from pathlib import Path
import pickle
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from janelia_core.dataprocessing.dataset import ROIDataset
from janelia_core.utils.data_saving import append_ts

## Parameters go here

In [3]:
ps = dict()

# Location of excel file specifying where the data for each experiment is saved relative to the base folder
ps['data_loc_file'] = r'A:\projects\keller_vnc\data\experiment_data_locations.xlsx'

# Subfolder containing the dataset for each subject
ps['dataset_folder'] = 'extracted'

# Base folder where datasets are stored 
ps['dataset_base_folder'] =r'K:\\SV4'

# Data to calculate Delta F/F for in each dataset
ps['f_ts_str'] = 'f_1_5_5'
ps['bl_ts_str'] = 'bl_1_5_5'

ps['n_smps'] = 3 # Number of samples to take before and after a manipulation 

# Parameters for calculating dff
ps['background'] = 100
ps['ep'] = 20

# Location to save results
ps['save_folder'] = r'A:\projects\keller_vnc\results\whole_brain_stats'
ps['save_str'] = 'dff_1_5_5_with_ep'

## Read in excel file specifying location of datasets

In [4]:
def c_fcn(str):
    return str.replace("'", "")
converters = {0:c_fcn, 1:c_fcn}

data_locs = pd.read_excel(ps['data_loc_file'], header=1, usecols=[1, 2], converters=converters)

## Helper function to calculate $\Delta F/F$

In [5]:
def calc_dff(f, b, background=ps['background'], ep=ps['ep']):
    return (f-b)/(b-background+ep)

## Extract dff along with behavior information from each dataset

In [6]:
n_datasets = len(data_locs)
event_annots = [None]*n_datasets

for d_i in range(n_datasets):
    # Get path to dataset
    dataset_path = (Path(ps['dataset_base_folder']) / data_locs['Main folder'][d_i] / data_locs['Subfolder'][d_i] / 
                    Path(ps['dataset_folder']) / '*.pkl')
    dataset_file = glob.glob(str(dataset_path))
    if len(dataset_file) != 0:
        dataset_file = dataset_file[0]

        # Load the dataset
        with open(dataset_file, 'rb') as f:
            dataset = ROIDataset.from_dict(pickle.load(f))
    
        # Calculate dff
        f=dataset.ts_data[ps['f_ts_str']]['vls'][:]
        b=dataset.ts_data[ps['bl_ts_str']]['vls'][:]
        dff = calc_dff(f=f, b=b)
        
        # Extract dff before and after each manipulation event and store it 
        event_annot = dataset.metadata['manip_event_annotations']
        n_events = len(event_annot)
        dff_before = [None]*n_events
        dff_after = [None]*n_events
        for e_i in range(n_events):
            m_start = event_annot['Manipulation Start'].to_numpy()[e_i]
            m_end = event_annot['Manipulation End'].to_numpy()[e_i]
            dff_before[e_i] = np.mean(dff[m_start-3:m_start,:], axis=0)
            dff_after[e_i] = np.mean(dff[m_end+1:m_end+4,:], axis=0)

        event_annot['dff_before'] = dff_before
        event_annot['dff_after'] = dff_after
    
        # Save results
        event_annots[d_i] = event_annot
    
        # Give user some feedback
        print('Done processing dataset ' + str(d_i + 1) + ' of ' + str(n_datasets) + '.')

# Place event annotations into a single table
event_annots = pd.concat(event_annots)

Done processing dataset 1 of 66.
Done processing dataset 2 of 66.
Done processing dataset 3 of 66.
Done processing dataset 4 of 66.
Done processing dataset 5 of 66.
Done processing dataset 6 of 66.
Done processing dataset 7 of 66.
Done processing dataset 8 of 66.
Done processing dataset 9 of 66.
Done processing dataset 10 of 66.
Done processing dataset 11 of 66.
Done processing dataset 12 of 66.
Done processing dataset 13 of 66.
Done processing dataset 14 of 66.
Done processing dataset 15 of 66.
Done processing dataset 16 of 66.
Done processing dataset 17 of 66.
Done processing dataset 18 of 66.
Done processing dataset 19 of 66.
Done processing dataset 21 of 66.
Done processing dataset 22 of 66.
Done processing dataset 23 of 66.
Done processing dataset 24 of 66.
Done processing dataset 25 of 66.
Done processing dataset 27 of 66.
Done processing dataset 28 of 66.
Done processing dataset 29 of 66.
Done processing dataset 30 of 66.
Done processing dataset 31 of 66.
Done processing dataset

In [8]:
## Save results
rs = dict()
rs['ps'] = ps
rs['event_annots'] = event_annots

save_name = append_ts(ps['save_str']) + '.pkl'
save_path = Path(ps['save_folder']) / save_name
with open(save_path, 'wb') as f:
    pickle.dump(rs, f)
    
print('Saved results to: ' + str(save_path))

Saved results to: A:\projects\keller_vnc\results\whole_brain_stats\dff_1_5_5_with_ep_2020_01_20_15_52_29_105309.pkl


In [ ]:
f=dataset.ts_data[ps['f_ts_str']]['vls'][:]
b=dataset.ts_data[ps['bl_ts_str']]['vls'][:]

In [ ]:
np.min(b)

In [ ]:
(1 - 1/52)**52

In [7]:
event_annots

,Date and sample,Manipulation Start,Manipulation End,Precede Behavior,Time differ between end of PB from Stimulus ONSET,Time differ between start of PB from Stimulus ONSET,Succeed Behavior,Time difference between start of SB from Stimus ONSET,Interval Time,Transtion Time,dff_before,dff_after
0,CW_17-08-23-L1,216,220,forward,1.0,34.0,back hunch,12.0,0.4010,4.8120,"[0.054598767, 0.06078657, 0.03721669, 0.038682...","[0.048961718, 0.069584034, 0.03722217, 0.01916..."
1,CW_17-08-23-L1,1795,1799,forward,2.0,18.0,back hunch,10.0,0.4010,4.0100,"[0.03240184, 0.044733804, 0.024001548, 0.02218...","[0.036694378, -0.010718676, 0.025001233, 0.004..."
2,CW_17-08-23-L2,114,119,forward,1.0,4.0,backward,6.0,0.3634,2.1804,"[0.04760842, 0.080916405, 0.06997735, 0.038695...","[0.07838976, 0.093723916, 0.11073011, 0.063612..."
3,CW_17-08-23-L2,2471,2476,forward,4.0,19.0,backward,6.0,0.3634,2.1804,"[0.014568833, 0.018858666, 0.015992953, 0.0200...","[0.044470083, 0.04860921, 0.0775243, 0.0778435..."
4,CW_17-08-23-L2,4644,4649,forward,5.0,10.0,backward,5.0,0.3634,1.8170,"[0.05529377, 0.004222768, 0.028083421, 0.04188...","[0.14517678, 0.12293305, 0.1137105, 0.08082393..."
...,...,...,...,...,...,...,...,...,...,...,...,...
273,CW_17-12-11-L3,2724,2729,quiet,NaN,NaN,forward,6.0,0.3756,2.2536,"[0.044009406, 0.035009526, 0.06686162, 0.08253...","[0.06621941, 0.07362655, 0.051485617, 0.078565..."
274,CW_17-12-11-L3,3445,3450,quiet,NaN,NaN,forward,9.0,0.3756,3.3804,"[0.028829338, 0.019474708, 0.049133588, 0.0383...","[0.064021826, 0.076009095, 0.066019125, 0.0738..."
275,CW_17-12-11-L3,4144,4149,quiet,NaN,NaN,forward,9.0,0.3756,3.3804,"[0.014759668, 0.010661985, 0.03287143, 0.00906...","[0.030296793, 0.08557811, 0.091722935, 0.08743..."
276,CW_17-12-11-L3,4845,4850,quiet,NaN,NaN,forward,10.0,0.3756,3.7560,"[0.0056119356, -0.0070766318, 0.028791232, 0.0...","[0.052880257, 0.048214655, 0.0755431, 0.049634..."
